In [1]:
import os
import json
import networkx as nx
import pandas as pd
from google.cloud import bigquery
import requests
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from pathlib import Path

part_path = Path("part-2")
part_path.mkdir(exist_ok=True)

raw_path = Path(f"{part_path}/raw")
raw_path.mkdir(exist_ok=True)

processed_path = Path(f"{part_path}/processed")
processed_path.mkdir(exist_ok=True)

submission_path = Path(f"{part_path}/submission")
submission_path.mkdir(exist_ok=True)

In [3]:
print(raw_path)

part-2/raw


In [4]:
# replace with your path to credentials
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../oso_gcp_credentials.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.getenv('GCP_CREDENTIAL_PATH')
# replace with your project name
# client = bigquery.Client(project='opensource-observer')
client = bigquery.Client(project=os.getenv('GCP_PROJECT_ID'))

In [5]:
def stringify_array(arr):
    return "'" + "','".join(arr) + "'"

In [6]:
def remove_str(projects):
    projects = [p.replace('https://github.com/', '') for p in projects]
    return projects

In [7]:
def remove_str_df(df):
    df['project_a'] = df['project_a'].str.replace('https://github.com/', '')
    df['project_b'] = df['project_b'].str.replace('https://github.com/', '')
    return df


In [8]:
def get_dataset():
    if str(part_path) == 'part-1':
        # repository_url = (
        #     "https://raw.githubusercontent.com/deepfunding/mini-contest/refs/heads/main/"
        # )
        
        # df_train = pd.read_csv(f"{repository_url}/dataset.csv")
        # df_test = pd.read_csv(f"{repository_url}/test.csv")
        # df_train.to_csv(f"{part_path}/train.csv", index=False)
        # df_test.to_csv(f"{part_path}/test.csv", index=False)
        df_train = pd.read_csv(f"{part_path}/train.csv")
        df_test = pd.read_csv(f"{part_path}/test.csv")
        
    elif str(part_path) == 'part-2':
        df_train = pd.read_csv(f"{part_path}/train.csv")
        df_test = pd.read_csv(f"{part_path}/test.csv")
     
    return df_train, df_test

In [9]:
df_train, df_test = get_dataset()

In [10]:
len(df_train), len(df_test)

(20958, 4261)

In [11]:
projects = pd.concat([
    df_train['project_a'],
    df_train['project_b'],
    df_test['project_a'],
    df_test['project_b']
]).unique().tolist()

In [12]:
projects[:5]

['https://github.com/mochajs/mocha',
 'https://github.com/chzyer/readline',
 'https://github.com/gulpjs/gulp',
 'https://github.com/webpack/webpack',
 'https://github.com/redux-saga/redux-saga']

In [13]:
len(projects)

117

In [34]:
def get_repo_data(projects):
    headers = {
    "Accept": "application/vnd.github+json",
    "Authorization": f"Bearer {os.getenv('GITHUB_TOKEN')}",
    "X-GitHub-Api-Version": "2022-11-28",
    }

    data = []
    for project in projects:
        api_url = f"https://api.github.com/repos/{project}"
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()
        data.append(response.json())
    df_projects = pd.DataFrame(data)
    df_projects.to_csv(f"{raw_path}/github-projects.csv", index=False)

    return df_projects

In [35]:
trimed_projects = remove_str(projects)
df_projects = get_repo_data(trimed_projects)


In [36]:
def get_repo_activity(urls):
    
    q = f"""
    WITH repos as(
    SELECT
        artifact_id,
        artifact_url AS repo_url,
    FROM
        `oso_production.repositories_v0`
    ),
    events as(
    SELECT
        artifact_id,
        event_type,
        CAST(sum(amount) AS FLOAT64) AS total_amount
    FROM
        `oso_production.events_monthly_to_artifact`
    WHERE bucket_month >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 180 DAY)
    GROUP BY 1,2
    )
    SELECT
        repos.repo_url,
        events.event_type,
        events.total_amount
    FROM repos
    JOIN events
        ON repos.artifact_id = events.artifact_id
    WHERE repo_url IN ({stringify_array(urls)})

    """

    results = client.query(q)
    df = results.to_dataframe()
    df_pivot = df.pivot_table(index=['repo_url'], columns='event_type', values='total_amount', fill_value=0)
    df_pivot.reset_index(inplace=True)
    df_pivot.to_csv(f"{raw_path}/github-activity.csv", index=False)
    return df_pivot

In [37]:
df_act = get_repo_activity(projects)

/home/jake/deepfunding/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [38]:
with open("unweighted_graph.json", 'r') as f:
    graph_data = json.load(f)

G_original = nx.node_link_graph(graph_data)
print("Nodes:", len(G_original.nodes))
repo_urls = [x for x in G_original.nodes]
print("Edges:", len(G_original.edges))

Nodes: 5235
Edges: 17367


/home/jake/deepfunding/.venv/lib/python3.13/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


In [39]:
def get_repo_metrics_and_metadata(urls):
    query = f"""
    WITH repos AS (
    SELECT *
    FROM `oso_production.repositories_v0`
    ),
    package_owners AS (
    SELECT
        package_owner_artifact_id,
        package_artifact_source,
        package_artifact_name,
        CONCAT(package_artifact_source, '/', package_artifact_name) AS package_tag
    FROM `oso_production.package_owners_v0`
    WHERE package_owner_artifact_id IN (SELECT artifact_id FROM repos)
    ),
    oso_dependents AS (
    SELECT
        package_owners.package_owner_artifact_id,
        COUNT(DISTINCT package_owners.package_tag) AS num_packages,
        COUNT(DISTINCT sboms.from_artifact_namespace) AS num_dependents_in_oso,
        ARRAY_AGG(DISTINCT package_owners.package_tag) AS list_of_packages,
        ARRAY_AGG(DISTINCT sboms.from_artifact_namespace) AS list_of_dependents_in_oso
    FROM `oso_production.sboms_v0` AS sboms
    JOIN package_owners
        ON sboms.to_package_artifact_name = package_owners.package_artifact_name
        AND sboms.to_package_artifact_source = package_owners.package_artifact_source
    GROUP BY 1
    ),
    grants AS (
    SELECT
        funding.to_project_id AS project_id,
        ARRAY_AGG(DISTINCT projects.display_name) AS list_of_funders,
        SUM(funding.amount) AS total_funding_usd,
        SUM(CASE WHEN funding.time > '2023-01-01' THEN funding.amount ELSE 0 END) AS total_funding_usd_since_2023
    FROM `oso_production.oss_funding_v0` AS funding
    JOIN `oso_production.projects_v1` AS projects
        ON funding.from_project_id = projects.project_id
    WHERE funding.from_project_name IN ('gitcoin', 'octant-golemfoundation', 'opencollective', 'optimism')
    GROUP BY 1
    ),
    combined AS (
    SELECT
        repos.artifact_url AS repo_url,
        repos.artifact_namespace AS maintainer,
        repos.language,
        repos.is_fork,
        DATE(repos.created_at) as created_at,
        DATE(repos.updated_at) as updated_at,
        repos.star_count,
        repos.fork_count,
        COALESCE(oso_dependents.num_packages, 0) AS num_packages,
        COALESCE(oso_dependents.num_dependents_in_oso, 0) AS num_dependents_in_oso,
        oso_dependents.list_of_dependents_in_oso,
        oso_dependents.list_of_packages,
        grants.list_of_funders,
        COALESCE(grants.total_funding_usd, 0) AS total_funding_usd,
        COALESCE(grants.total_funding_usd_since_2023, 0) AS total_funding_usd_since_2023
    FROM repos
    LEFT JOIN oso_dependents
        ON repos.artifact_id = oso_dependents.package_owner_artifact_id
    LEFT JOIN grants
        ON repos.project_id = grants.project_id
    )
    SELECT
    *,
    PERCENT_RANK() OVER (ORDER BY num_dependents_in_oso) AS oso_dependency_rank,
    COUNT(*) OVER (PARTITION BY language) AS num_repos_in_same_language,
    PERCENT_RANK() OVER (PARTITION BY language ORDER BY num_dependents_in_oso) AS oso_dependency_rank_for_language
    FROM combined
    WHERE repo_url IN ({stringify_array(repo_urls)})

    """

    # execute the query and save it
    results = client.query(query)
    df = results.to_dataframe()
    df.to_csv(f"{raw_path}/repo_metrics_and_metadata.csv")
    print("Query saved to local csv file.")
 
    return df


In [40]:
df_repo = get_repo_metrics_and_metadata(repo_urls)

/home/jake/deepfunding/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Query saved to local csv file.


In [41]:
def get_dependent_repos_in_OSO():
  query = """
  select
    p.project_id,
    pkgs.package_artifact_source,
    pkgs.package_artifact_name,
    count(distinct sboms.from_project_id) as num_dependents
  from `oso_production.package_owners_v0` pkgs
  join `oso_production.sboms_v0` sboms
    on pkgs.package_artifact_name = sboms.to_package_artifact_name
    and pkgs.package_artifact_source = sboms.to_package_artifact_source
  join `oso_production.projects_v1` p
    on pkgs.package_owner_project_id = p.project_id
  where pkgs.package_owner_project_id is not null
  group by 1,2,3
  order by 4 desc
  """

  results = client.query(query)
  df = results.to_dataframe()
  df.to_csv(f"{raw_path}/dependent-metrics.csv")
  print("Query saved to local csv file.")
  
  return df

In [42]:
df_dependent = get_dependent_repos_in_OSO()

/home/jake/deepfunding/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Query saved to local csv file.
